# **main.ipynb**

The purpose of this script is to statistically analyze the different card sets of the game *Magic: The Gathering* in order to:

1. Identify the most relevant cards in each set.
2. Compare the metadata associated with cards across different sets.

The goal of the analysis is to objectively determine the intrinsic value of a card *within* and *across* sets.

The intended application of this analysis is to leverage this data to improve performance in limited formats (*sealed, draft*).

# **Initialisation**

In [3]:
import os
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

from set_analyzer import *

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
%load_ext autoreload
%autoreload 2

Upload data from JSON (dataset from https://mtgjson.com/)

In [6]:
# Set path of the folder containing dataset
dataset_FolderPath = Path.cwd().parent / 'datasets' / 'MTG_datasets' # @dev TBC before each use

# Set path of the File
dataset_FileName = 'AllPrintings.json'
dataset_FilePath = dataset_FolderPath / dataset_FileName

In [7]:
# Load all datasets
data = pd.read_json(dataset_FilePath)
allSets = data.iloc[2:]['data'] # 2 first rows of JSON files are metadata

In [8]:
setCompare = allSets.apply(pd.Series)[['baseSetSize', 'code', 'totalSetSize', 'type', 'name', 'releaseDate']]

# **Analysis of one set** 

In [10]:
set_code = 'OTJ'
cards = loadLimitedSet(allSets, set_code)

## 1) SPEED

Format speed can be caracterized by :
- the ratio of creatures
- the median creature `manaValue`
- the median `powerToManaValue` : above 1: creatures hit hard, fast
- the board state (see section 2)
- the number of interactions (see section 3)

In [12]:
limitedCreatureRatio, meanCreatureMV, meanPowerToMV = analyzeSetSpeed(cards)

# Add values to setCompare
setCompare = setCompare.copy()
setCompare.at[set_code, 'limited_CreatureRatio'] = limitedCreatureRatio
setCompare.at[set_code, 'limited_meanCreatureManaValue'] = meanCreatureMV
setCompare.at[set_code, 'limited_meanCreaturePowerToManaValue'] = meanPowerToMV

## 2) BOARD STATE

- the mean creature `power`
- the mean creature `thougness`
- the mean `powerToToughness ratio`: above 1: creatures are likely to hit harder and defend badly (and vice versa)
- ratio of evasive creatures (ie. 'Flying', 'Trample', 'Menace')

In [14]:
meanCreaturePower, meanCreatureToughness, meanPowerToToughness, KWCount, evasiveKWCount = analyzeSetBoardState(cards)

# Add values to setCompare
setCompare = setCompare.copy()
setCompare.at[set_code, 'limited_meanCreaturePower'] = meanCreaturePower
setCompare.at[set_code, 'limited_meanCreatureToughness'] = meanCreatureToughness
setCompare.at[set_code, 'limited_meanCreaturePowerToToughness'] = meanPowerToToughness
setCompare.at[set_code, 'limited_KWCount'] = [KWCount]
setCompare.at[set_code, 'limited_evasiveKWCount'] = [evasiveKWCount]

## 3) FIXING

- monocolor-to-multicolor ratio (lands excluded)
- multi-pip ratio : cards with more that one colored pip in mana cost
- ratio of mana producer + types (lands, manadorks, manarocks, treasures)
- type of mana produced (TBD)

In [16]:
monocolorToMulticolorRatio, multiPipRatio, manaProducerRatio, nonLand_manaProducerRatio, manaProducerTypes = analyzeSetFixing(cards)

# Add values to setCompare
setCompare = setCompare.copy()
setCompare.at[set_code, 'limited_MonoToMulticolorRatio'] = monocolorToMulticolorRatio
setCompare.at[set_code, 'limited_MultiPipRatio'] = multiPipRatio
setCompare.at[set_code, 'limited_manaProducerRatio'] = manaProducerRatio
setCompare.at[set_code, 'limited_nonLand_manaProducerRatio'] = nonLand_manaProducerRatio
setCompare.at[set_code, 'limited_manaProducerTypes'] = [manaProducerTypes]

## 4) Interactions (TBD)

a quel point le set est interactif ?
000 - définir ce qu'est une interaction
- ratio de permanents
- pourcentage d'interaction
- la "vitesse" de l'interaction = distribution de mana value des sorts interactifs
- type d'interaction : single-target removal + combat trick
- color pie

In [18]:
"""
# ratio of permanents

def checklist(items_wanted, items_tbc):
  return any(item in items_wanted for item in items_tbc)

permanent_index = [item for item in type_index if (item !='Instant' and item!='Sorcery')]
cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==False)] #non-permanent
cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)]  #permanents

print('permanent ratio = ' + str(len(cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)])/len(cards)*100) + ' %')

# get interactive cards

interaction_list = [
    'destroy',
    'exile',
    'counter',
    'target'
]

def interactive_card(str):
  if any(word in str for word in interaction_list):
    return True
  else:
    return False

cards[cards['text'].apply(interactive_card)]
"""

"\n# ratio of permanents\n\ndef checklist(items_wanted, items_tbc):\n  return any(item in items_wanted for item in items_tbc)\n\npermanent_index = [item for item in type_index if (item !='Instant' and item!='Sorcery')]\ncards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==False)] #non-permanent\ncards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)]  #permanents\n\nprint('permanent ratio = ' + str(len(cards['types'][cards['types'].apply(lambda x: checklist(x,permanent_index)==True)])/len(cards)*100) + ' %')\n\n# get interactive cards\n\ninteraction_list = [\n    'destroy',\n    'exile',\n    'counter',\n    'target'\n]\n\ndef interactive_card(str):\n  if any(word in str for word in interaction_list):\n    return True\n  else:\n    return False\n\ncards[cards['text'].apply(interactive_card)]\n"

# **Comparing sets**

In [121]:
last5_sets = ['DFT', 'FDN', 'DSK', 'BLB', 'OTJ']
standard_sets = last5_sets + ['MKM', 'LCI', 'WOE', 'MOM', 'ONE', 'BRO', 'DMU']
nonstandard_sets = ['LTR', 'MH3', 'CMM']

#sets = last5_sets
sets = standard_sets
#sets = standard_sets + nonstandard_sets
#sets = allSets.index.to_list()

In [145]:
for set_code in tqdm(sets):   
    cards = loadLimitedSet(allSets, set_code)
    limitedCreatureRatio, meanCreatureMV, meanPowerToMV = analyzeSetSpeed(cards)
    meanCreaturePower, meanCreatureToughness, meanPowerToToughness, KWCount, evasiveKWCount = analyzeSetBoardState(cards)
    monocolorToMulticolorRatio, multiPipRatio, manaProducerRatio, nonLand_manaProducerRatio, manaProducerTypes = analyzeSetFixing(cards)

    # Add values to setCompare
    setCompare = setCompare.copy()
    setCompare.at[set_code, 'limited_CreatureRatio'] = limitedCreatureRatio
    setCompare.at[set_code, 'limited_meanCreatureManaValue'] = meanCreatureMV
    setCompare.at[set_code, 'limited_meanCreaturePowerToManaValue'] = meanPowerToMV
    setCompare.at[set_code, 'limited_meanCreaturePower'] = meanCreaturePower
    setCompare.at[set_code, 'limited_meanCreatureToughness'] = meanCreatureToughness
    setCompare.at[set_code, 'limited_meanCreaturePowerToToughness'] = meanPowerToToughness
    setCompare.at[set_code, 'limited_KWCount'] = [KWCount]
    setCompare.at[set_code, 'limited_evasiveKWCount'] = [evasiveKWCount]
    setCompare.at[set_code, 'limited_MonoToMulticolorRatio'] = monocolorToMulticolorRatio
    setCompare.at[set_code, 'limited_MultiPipRatio'] = multiPipRatio
    setCompare.at[set_code, 'limited_manaProducerRatio'] = manaProducerRatio
    setCompare.at[set_code, 'limited_nonLand_manaProducerRatio'] = nonLand_manaProducerRatio
    setCompare.at[set_code, 'limited_manaProducerTypes'] = [manaProducerTypes]

100%|██████████| 12/12 [00:00<00:00, 15.23it/s]


In [147]:
set_sample = setCompare.loc[sets].sort_values(by='releaseDate', ascending=False)

[{'Flying': 16, 'Trample': 2, 'Menace': 0}]

In [127]:
# Normalize with Z-score scaler (can also use Min-Max scaler)
scaler = StandardScaler()
set_sample_norm = set_sample.copy()
set_sample_norm[set_sample_norm.select_dtypes(include='float').columns] = scaler.fit_transform(
    set_sample_norm[set_sample_norm.select_dtypes(include='float').columns])

In [133]:
# Function to update the plot
def update_plot(column_name, highlight_index=None):
    # Get the column index from the column name
    column_index = set_sample_norm.columns.get_loc(column_name)
    
    # Get the sorted data for plotting
    s = set_sample_norm.iloc[:, column_index].sort_values(ascending=False)
    
    # Create the plot
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    
    # Plot all bars
    sns.barplot(x=s.index, y=s.values, ax=ax, color='blue')
    
    # Highlight the specific index if specified
    if highlight_index is not None and highlight_index in s.index:
        highlight_value = s[highlight_index]
        ax.bar(highlight_index, highlight_value, color='red')  # Change color to red for highlight
    
    # Set axis labels and rotate x-axis labels for better visibility
    ax.set_ylabel(column_name)
    plt.xticks(rotation=90)

    plt.show()

# Create a dropdown widget to select the column name
column_selector = widgets.Dropdown(
    options=set_sample_norm.columns.tolist(),  # Use column names instead of indices
    value=set_sample_norm.columns[0],  # Default to the first column
    description='Column:',
    style={'description_width': 'initial'}
)

# Create a text box or dropdown widget to select which index to highlight (optional)
highlight_selector = widgets.Text(
    value='',  # Default to empty (no highlight)
    description='Highlight Index:',
    style={'description_width': 'initial'}
)

# Display the widgets and update the plot dynamically
widgets.interactive(update_plot, column_name=column_selector, highlight_index=highlight_selector)


interactive(children=(Dropdown(description='Column:', options=('baseSetSize', 'code', 'totalSetSize', 'type', …